In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [17]:
# variable that I want to extract for each storm

folder = 'geopotential'

year = '1990'

var = 'ERA5_'+year+'_'+folder

In [18]:
#local_path = '/Users/fabienaugsburger/myCloud/Unil/Master/Thesis/Datasets/Curnagl/ERA5_1990_2m_dewpoint_temperature.nc'
local_path = '/Users/fabienaugsburger/Desktop/ERA5_1990_geopotential.nc'
servor_path = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'+folder+'/'+var+'.nc'
#servor_path = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/'+folder+'/'+var+'.nc'

dew_point_xr = xr.open_dataset(local_path)
dew_point_xr = dew_point_xr.sel(level=500)

specific_var = list(dew_point_xr.variables)[4]
dew_point_xr

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 2920)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 500
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-12-31T21:00:00
Data variables:
    z          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-21 10:31:22 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [19]:
# import 1st track

#head = ['lon_east', 'lon_west', 'lat_south', 'lat_north']
#track_1 = pd.read_csv('tc_irad_tracks/tc_irad_1.txt')
#track_1.columns = head

# import all tracks

dates = pd.read_csv('storms_start_end.csv', sep=',')
dates['year'] = dates['start_date'].str[:4]

length_year = dates[dates['year'] == year].shape[0]
index_year = dates[dates['year'] == year].index[0]

for i in range(index_year,index_year+length_year):
    locals()['track_' + str(i+1)] = pd.read_csv('tc_irad_tracks/tc_irad_' + str(i+1) + '.txt')
    

In [20]:
# slice the time dimension

'''
start_first = dates['start_date'][0]
end_first = dates['end_date'][0]
'''
for i in range(index_year,index_year + length_year):
    start_temp = dates['start_date'][i]
    end_temp = dates['end_date'][i]

    # Create a new dataset for each iteration
    new_dataset = xr.Dataset({
        specific_var: dew_point_xr[specific_var],
        # Add other variables as needed
    })

    # Optionally, you can update the time dimension for the new dataset
    new_dataset = new_dataset.sel(time=slice(start_temp, end_temp))

    # Dynamically name the dataset variable
    locals()[f"dew_point_xr_{i}"] = new_dataset

'''
dew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))
dew_point_xr
'''

'\ndew_point_xr = dew_point_xr.sel(time=slice(start_first, end_first))\ndew_point_xr\n'

In [21]:
dew_point_xr_0

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 62)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 500
  * time       (time) datetime64[ns] 1990-01-21T03:00:00 ... 1990-01-28T18:00:00
Data variables:
    z          (time, latitude, longitude) float32 ...

In [6]:
# create new pd dataframe
'''
first_slice = track_1.iloc[0]

test_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))
test_dp'''


"\nfirst_slice = track_1.iloc[0]\n\ntest_dp     = dew_point_xr.sel(longitude=slice(first_slice['lon_east'], first_slice['lon_west']), latitude=slice(first_slice['lat_north'], first_slice['lat_south']))\ntest_dp"

In [7]:
# plot the first slice with matplotlib
'''
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 5))
plt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')'''

"\nimport matplotlib.pyplot as plt\n\n\nplt.figure(figsize=(10, 5))\nplt.pcolormesh(test_dp.longitude, test_dp.latitude, test_dp.d2m[0,:,:], cmap='coolwarm')"

In [8]:
# check if it's correct
'''
lon_test = np.asanyarray(dew_point_xr.longitude[:])
lat_test = np.asanyarray(dew_point_xr.latitude[:])


closest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()
closest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()
closest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()
closest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()

lon_test[closest_lon_w]
lat_test[closest_lat_s]
'''
# works

"\nlon_test = np.asanyarray(dew_point_xr.longitude[:])\nlat_test = np.asanyarray(dew_point_xr.latitude[:])\n\n\nclosest_lon_w =np.abs(lon_test - first_slice['lon_west']).argmin()\nclosest_lon_e = np.abs(lon_test - first_slice['lon_east']).argmin()\nclosest_lat_s = np.abs(lat_test - first_slice['lat_south']).argmin()\nclosest_lat_n = np.abs(lat_test - first_slice['lat_north']).argmin()\n\nlon_test[closest_lon_w]\nlat_test[closest_lat_s]\n"

In [9]:
#slice with new boundaries
'''
plt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')
plt.colorbar()'''

"\nplt.pcolormesh(np.flipud(dew_point_xr['d2m'][0,176:208,1157:1189]), cmap='coolwarm')\nplt.colorbar()"

In [10]:
#lat_test[208]

In [22]:
# iterate through each time step

var_out = []
'''
for i in range(0, len(track_1)):
    lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_1.iloc[i]

    lon_test = np.asanyarray(dew_point_xr.longitude[:])
    lat_test = np.asanyarray(dew_point_xr.latitude[:])

    closest_lon_w =np.abs(lon_test - lon_w_temp).argmin()
    closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
    closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
    closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

    temp_ds = np.asarray(dew_point_xr['d2m'])

    var_out.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
'''
var_out
for j in range (index_year,index_year + length_year):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []
    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        closest_lon_w_coor = lon_test[closest_lon_w]
        closest_lon_e_coor = lon_test[closest_lon_e]
        closest_lat_s_coor = lat_test[closest_lat_s]
        closest_lat_n_coor = lat_test[closest_lat_n]

        # Use .roll to handle the 0°/360° boundary

        if closest_lon_w_coor < 100 and closest_lon_e_coor > 100:
            roll_shift = {'longitude':int(round(closest_lon_w_coor,0)), 'longitude': int(round(closest_lon_e_coor,0))}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)
        else:
            dew_point_temp_rolled = dew_point_temp
        # Slice the dataset based on the rolled longitudes and latitudes
        temp_ds_time = dew_point_temp_rolled[specific_var].isel(time=i)
        temp_ds = temp_ds_time.sel(latitude=slice(closest_lat_n_coor, closest_lat_s_coor),
                                   longitude=slice(closest_lon_e_coor, closest_lon_w_coor)).values

        #temp_ds = np.asarray(dew_point_temp['d2m'])

        #var_out_temp.append(temp_ds[i,closest_lat_n:closest_lat_s,closest_lon_e:closest_lon_w])
        var_out_temp.append(temp_ds)

    locals()[f"var_out_{j+1}"] = var_out_temp


In [12]:
# test

'''for j in range(0, 4):
    track_temp = pd.read_csv(f'tc_irad_tracks/tc_irad_{j+1}.txt')
    dew_point_temp = locals()[f"dew_point_xr_{j}"]
    var_out_temp = []

    for i in range(0, len(track_temp)):
        lon_e_temp, lon_w_temp, lat_s_temp, lat_n_temp = track_temp.iloc[i]

        lon_test = np.asanyarray(dew_point_temp.longitude[:])
        lat_test = np.asanyarray(dew_point_temp.latitude[:])

        closest_lon_w = np.abs(lon_test - lon_w_temp).argmin()
        closest_lon_e = np.abs(lon_test - lon_e_temp).argmin()
        closest_lat_s = np.abs(lat_test - lat_s_temp).argmin()
        closest_lat_n = np.abs(lat_test - lat_n_temp).argmin()

        # Check if the indices create a valid slice
        if closest_lat_s > closest_lat_n and closest_lon_w < closest_lon_e:
            # Use .roll to handle the 0°/360° boundary
            roll_shift = {'longitude': closest_lon_e, 'longitude': closest_lon_w}
            dew_point_temp_rolled = dew_point_temp.roll(roll_shift, roll_coords=True)

            # Slice the dataset based on the rolled longitudes and latitudes
            temp_ds = dew_point_temp_rolled['d2m'].isel(time=i, latitude=slice(closest_lat_n, closest_lat_s),
                                                        longitude=slice(closest_lon_e, closest_lon_w)).values

            var_out_temp.append(temp_ds)
        else:
            print(f"Invalid indices for square at time index {i} in track {j+1}")

    locals()[f"var_out_{j+1}"] = var_out_temp
'''
'''
roll_test = {'longitude': 10, 'longitude': 350}

test_ds_rolled = dew_point_xr.roll(roll_test, roll_coords=True)

test_ds_rolled

test_ds = test_ds_rolled['d2m'].sel(latitude=slice(80, 40), longitude=slice(350.25, 10.25))
test_ds

test_ds = test_ds_rolled['d2m'].isel(time=0)
test_ds
'''

<xarray.DataArray 'd2m' (latitude: 721, longitude: 1440)>
array([[242.3083 , 242.3083 , 242.3083 , ..., 242.3083 , 242.3083 ,
        242.3083 ],
       [242.78452, 242.78452, 242.78629, ..., 242.781  , 242.781  ,
        242.78276],
       [243.4769 , 243.47865, 243.48216, ..., 243.46635, 243.47162,
        243.47337],
       ...,
       [245.03384, 245.0356 , 245.03912, ..., 245.02155, 245.02505,
        245.02858],
       [245.15686, 245.16037, 245.16212, ..., 245.14807, 245.15158,
        245.1551 ],
       [245.23593, 245.23593, 245.23593, ..., 245.23593, 245.23593,
        245.23593]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 272.5 272.8 273.0 273.2 ... 271.8 272.0 272.2
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    time       datetime64[ns] 1990-01-01
Attributes:
    units:      K
    long_name:  2 metre dewpoint temperature

In [101]:
#print(closest_lon_w, closest_lon_e, closest_lat_s, closest_lat_n)
temp_ds

array([[273.3718 , 273.05548, 272.83408, ..., 271.04868, 271.09964,
        270.90985],
       [273.38058, 273.12402, 272.89734, ..., 272.06085, 272.20496,
        272.27173],
       [273.38937, 273.21716, 272.92368, ..., 273.86383, 273.84274,
        273.64243],
       ...,
       [272.9694 , 272.2489 , 271.22263, ..., 267.39175, 266.26886,
        265.38318],
       [272.08722, 271.28766, 270.5408 , ..., 266.27588, 265.54135,
        265.276  ],
       [271.81308, 271.0276 , 270.54257, ..., 265.73114, 265.40076,
        265.03174]], dtype=float32)

In [15]:
# check if each square has the same shape

for i in range(0, len(var_out_7)):
    print(var_out_7[i].shape)

(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)
(33, 33)


In [41]:
var_out_7[15]

array([[275.39618, 275.18005, 274.9604 , ..., 278.37302, 278.58566,
        278.71747],
       [275.76874, 275.56488, 275.26086, ..., 278.3045 , 278.45035,
        278.56107],
       [275.85132, 275.36456, 275.11502, ..., 278.2342 , 278.44333,
        278.61203],
       ...,
       [280.64343, 280.89297, 281.14777, ..., 285.89246, 285.93814,
        286.07343],
       [280.9615 , 281.18994, 281.4465 , ..., 286.07697, 286.2527 ,
        286.29663],
       [281.29538, 281.50977, 281.76282, ..., 286.21228, 286.2404 ,
        286.17184]], dtype=float32)

In [23]:
# mean, min, max value of each time step

mean_out = []
min_out = []
max_out = []
sigma_out = []
skweness_out = []
kurto_out = []

stats_list = []
all_stats = []
for j in range(index_year, index_year + length_year):

    var_out = locals()[f"var_out_{j+1}"]
    mean_out_temp = []
    min_out_temp = []
    max_out_temp = []
    sigma_out_temp = []
    skweness_out_temp = []
    kurto_out_temp = []

    for i in range(0, len(var_out)):
        mean_out_temp.append(np.mean(var_out[i]))
        min_out_temp.append(np.min(var_out[i]))
        max_out_temp.append(np.max(var_out[i]))
        sigma_out_temp.append(np.std(var_out[i]))
        skweness_out_temp.append(pd.Series(np.asarray(var_out[i]).reshape(-1)).skew())
        kurto_out_temp.append(pd.Series(np.asarray(var_out[i]).reshape(-1)).kurtosis())
    
    mean_out.append(mean_out_temp)
    min_out.append(min_out_temp)
    max_out.append(max_out_temp)
    sigma_out.append(sigma_out_temp)
    skweness_out.append(skweness_out_temp)
    kurto_out.append(kurto_out_temp)

    locals()[f"mean_out_{j+1}"] = mean_out_temp
    locals()[f"min_out_{j+1}"] = min_out_temp
    locals()[f"max_out_{j+1}"] = max_out_temp
    locals()[f"sigma_out_{j+1}"] = sigma_out_temp
    locals()[f"skweness_out_{j+1}"] = skweness_out_temp
    locals()[f"kurto_out_{j+1}"] = kurto_out_temp

In [10]:
# save as csv

for j in range(index_year,index_year+length_year):

    locals()[f"mean_out_{j+1}"] = pd.DataFrame(locals()[f"mean_out_{j+1}"])
    locals()[f"min_out_{j+1}"] = pd.DataFrame(locals()[f"min_out_{j+1}"])
    locals()[f"max_out_{j+1}"] = pd.DataFrame(locals()[f"max_out_{j+1}"])
    locals()[f"sigma_out_{j+1}"] = pd.DataFrame(locals()[f"sigma_out_{j+1}"])
    locals()[f"skweness_out_{j+1}"] = pd.DataFrame(locals()[f"skweness_out_{j+1}"])
    locals()[f"kurto_out_{j+1}"] = pd.DataFrame(locals()[f"kurto_out_{j+1}"])

    locals()[f"mean_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/mean_{j+1}.csv')
    locals()[f"min_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/min_{j+1}.csv')
    locals()[f"max_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/max_{j+1}.csv')
    locals()[f"sigma_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/sigma_{j+1}.csv')
    locals()[f"skweness_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/skweness_{j+1}.csv')
    locals()[f"kurto_out_{j+1}"].to_csv(f'datasets/{folder}/storm_{j+1}/kurto_{j+1}.csv')

'''df_mean = pd.DataFrame(mean_out)
df_min = pd.DataFrame(min_out)
df_max = pd.DataFrame(max_out)
df_sigma = pd.DataFrame(sigma_out)

df_mean.to_csv(var+'_mean.csv')
df_min.to_csv(var+'_min.csv')
df_max.to_csv(var+'_max.csv')
df_sigma.to_csv(var+'_sigma.csv')'''

"df_mean = pd.DataFrame(mean_out)\ndf_min = pd.DataFrame(min_out)\ndf_max = pd.DataFrame(max_out)\ndf_sigma = pd.DataFrame(sigma_out)\n\ndf_mean.to_csv(var+'_mean.csv')\ndf_min.to_csv(var+'_min.csv')\ndf_max.to_csv(var+'_max.csv')\ndf_sigma.to_csv(var+'_sigma.csv')"

For all variables, (class by variable (folder), then by storm)) 